In [ ]:
# Setup
%pip install -e '../../.[kafka-python]'

import json
import uuid
from datetime import datetime
from diaspora_event_sdk import Client as GlobusClient
from diaspora_event_sdk.sdk.kafka_client import KafkaProducer, KafkaConsumer

# os.environ["DIASPORA_SDK_ENVIRONMENT"] = "local"
c = GlobusClient()

In [ ]:
# Display current user subject
print(f"Current user subject: {c.subject_openid}")

In [ ]:
# 1. Create User (Optional)
# create_user is not strictly necessary — create_key will automatically
# create the user on the service side if they don't already exist.
user_result = c.create_user()
print(json.dumps(user_result, indent=2, default=str))

In [ ]:
# Display namespace (automatically computed from subject)
print(f"Using namespace: {c.namespace}")

In [ ]:
# 2. Create Key
# Creates an IAM access key for Kafka authentication.
# Also creates the user and namespace if they don't exist yet.
# Returns access_key, secret_key, endpoint, and create_date.
key_result = c.create_key()
print(json.dumps(key_result, indent=2, default=str))

In [ ]:
# 3. List Namespaces
# Lists all namespaces owned by the user and their topics.
namespaces_result = c.list_namespaces()
print(json.dumps(namespaces_result, indent=2, default=str))

In [ ]:
# 4. Create Topic
# Creates a topic under the user's namespace.
# The Kafka topic name will be "{namespace}.{topic_name}".
topic_name = f"topic-{str(uuid.uuid4())[:5]}"
create_topic_result = c.create_topic(topic_name)
print(json.dumps(create_topic_result, indent=2, default=str))

In [ ]:
# Kafka topic name format: "{namespace}.{topic}"
kafka_topic = f"{c.namespace}.{topic_name}"
print(f"Kafka topic name: {kafka_topic}")

In [ ]:
# 5. Produce Messages
# KafkaProducer auto-configures authentication and serialization.
# Messages are JSON-serialized by default.
p = KafkaProducer(kafka_topic)
for i in range(3):
    message = {
        "message_id": i + 1,
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "content": f"Message {i + 1} from v3 API",
    }
    future = p.send(kafka_topic, message)
    result = future.get(timeout=30)
    print(f"Produced message {i + 1}: offset={result.offset}")
p.close()

In [ ]:
# 6. Consume Messages
# KafkaConsumer auto-configures authentication.
# Use auto_offset_reset="earliest" to read from the beginning of the topic.
consumer = KafkaConsumer(kafka_topic, auto_offset_reset="earliest")
messages = consumer.poll(timeout_ms=10000)
for tp, msgs in messages.items():
    for message in msgs:
        data = json.loads(message.value.decode("utf-8"))
        print(f"Consumed: {data}")
consumer.close()

In [ ]:
# 7. Recreate Topic
# Deletes and recreates the topic, clearing all messages.
recreate_result = c.recreate_topic(topic_name)
print(json.dumps(recreate_result, indent=2, default=str))

In [ ]:
# 8. Delete Topic
delete_topic_result = c.delete_topic(topic_name)
print(json.dumps(delete_topic_result, indent=2, default=str))

In [ ]:
# 9. Delete Key
# Only removes the IAM access key — topics, namespace, and user are preserved.
# A new key can be created later with create_key().
delete_key_result = c.delete_key()
print(json.dumps(delete_key_result, indent=2, default=str))

In [ ]:
# 10. Delete User
# Full cleanup: deletes the IAM user, access keys, policies, all topics,
# and the namespace. This is irreversible — all data is lost.
delete_user_result = c.delete_user()
print(json.dumps(delete_user_result, indent=2, default=str))